In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import env
from prepare_zillow import pool_nulls
from prepare_zillow import wrangle_zillow
from prepare_zillow import null_dropper
from prepare_zillow import handle_outliers
from prepare_zillow import rename_cols
from prepare_zillow import fips_name
from prepare_zillow import dummy_var

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split
from env import host, user, password
import acquire_zillow

In [2]:
df = acquire_zillow.get_zillow_data()
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,NaN,3.50,4.00,None,NaN,3.50,NaN,...,"60,590,630,072,012.00",0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,NaN,1.00,2.00,None,NaN,1.00,NaN,...,"61,110,010,023,006.00",0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,14186244,NaN,NaN,NaN,2.00,3.00,None,NaN,2.00,NaN,...,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,...,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,12095076,1.00,NaN,NaN,3.00,4.00,None,9.00,3.00,NaN,...,"60,374,608,001,014.00",-0.00,2017-01-01,Central,None,None,Central,Single Family Residential,None,None


In [3]:
df = wrangle_zillow(df)
df.head()

,parcelid,bathroom,bedroom,bed_bath,sqft,finished_sqft,fips,fullbathcnt,latitude,longitude,...,tax_amount,censustractandblock,logerror,transaction_date,property_desc,county,has_pool,county_Los Angeles,county_Orange_county,county_Ventura_county
0,14297519,3.50,4.00,3.50,"3,100.00","3,100.00",6059.0,3.00,33.63,-117.87,...,"11,013.72","60,590,630,072,012.00",0.03,2017-01-01,Single Family Residential,Orange_county,0.0,0,1,0
1,17052889,1.00,2.00,1.00,"1,465.00","1,465.00",6111.0,1.00,34.45,-119.28,...,"5,672.48","61,110,010,023,006.00",0.06,2017-01-01,Single Family Residential,Ventura_county,0.0,0,0,1
2,14186244,2.00,3.00,2.00,"1,243.00","1,243.00",6059.0,2.00,33.89,-117.82,...,"6,488.30","60,590,218,022,012.00",0.01,2017-01-01,Single Family Residential,Orange_county,1.0,0,1,0
3,12177905,3.00,4.00,3.00,"2,376.00","2,376.00",6037.0,3.00,34.25,-118.24,...,"1,777.51","60,373,001,001,006.00",-0.10,2017-01-01,Single Family Residential,Los Angeles,1.0,1,0,0
4,12095076,3.00,4.00,3.00,"2,962.00","2,962.00",6037.0,3.00,34.15,-118.18,...,"9,516.26","60,374,608,001,014.00",-0.00,2017-01-01,Single Family Residential,Los Angeles,1.0,1,0,0


In [ ]:
df.poolcnt.describe

In [ ]:
train, test = train_test_split(df,test_size=0.2, random_state=42)
train, validate = train_test_split(train,test_size=0.25, random_state=42)

In [ ]:
train.head()

In [ ]:
trafig, axs = plt.subplots(1, 3, figsize=(15, 7))

for ax, col in zip(axs, train.select_dtypes('number')):
    train[col].plot.hist(ax=ax, title=col, ec='black')

In [ ]:
import seaborn as sns
# overall logerror distribution 
sns.distplot(df.logerror)
plt.title('Log Error Distribution', fontsize=20)
plt.xlabel('Log Error')

plt.show()

log error looks normally distributed 

In [ ]:

plt.figure(figsize=(14, 10))
sns.heatmap(train.corr(), cmap='Greens', annot=True)
plt.title('Correlation Heatmap of All Features Including the Target', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(14,8))
with sns.color_palette('Blues'):
    sns.barplot(x='bathroom', y='logerror', data=train)
plt.xlabel('Bathroom Count')
plt.ylabel('Log Error')
plt.title('Does bathroom count impact log error?')
plt.show()

homes with smaller bathroom count tend to have smaller log error 

In [ ]:
plt.figure(figsize=(14,8))
with sns.color_palette("Blues"):
    sns.barplot(x='bedroom', y='logerror', data=train)
plt.xlabel('Bedroom Count')
plt.ylabel('Log Error')
plt.title('Does bedroom count impact log error?')
plt.show()

bedroom shows an increase in log error at higher counts 

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x='sqft', y='logerror', data=train, alpha=.4)
plt.xlabel('Finished Square Feet')
plt.ylabel('Log Error')
plt.title('Does sqft of a home impact log error?')
plt.show()

In [ ]:
plt.scatter(train.logerror,train.home_value)
plt.xlabel('Logerror')
plt.ylabel('Home Value')
plt.title('Logerror vs Home Value')

In [ ]:
train.columns

In [ ]:
train['regionidzip'].value_counts()